The first step will be to __trim/clean our raw Illumina data__.

In [1]:
!mkdir trimming

In [2]:
cd trimming

/home/working/12S/trimming


Prepare a text file specifying the samples to be processed including the format and location of the reads. 

The below command expects the Illumina data to be present in 2 fastq files (forward and reverse reads) per sample in a directory `../../raw_reads/`. It expects the files to be named 'sampleID-marker', followed by '\_1' or '\_2' to identify the forward/reverse read file respectively. sampleID must corresponds to the first column in the file `Sample_accessions.tsv`, marker is either '12S' or 'CytB'. 


The raw data need to be downloaded with `How_to_download_Rawdata_from_SRA.ipynb` (see [here](https://github.com/HullUni-bioinformatics/Handley_et_al_2018/blob/master/How_to_download_Rawdata_from_SRA.ipynb))

SampleID must corresponds to the first column in the file `Sample_accessions.tsv` (see [here](https://github.com/HullUni-bioinformatics/Handley_et_al_2018/blob/master/supplementary_data/Sample_accessions.tsv))

In [3]:
%%bash

for a in $(cat ../../supplementary_data/Sample_accessions.tsv | grep "12S" | cut -f 1 | grep "SampleID" -v)
do
    R1=$(ls -1 ../../raw_reads/$a-12S_* | grep "_R1.fastq")
    R2=$(ls -1 ../../raw_reads/$a-12S_* | grep "_R2.fastq")

    echo -e "$a\tfastq\t$R1\t$R2"
done > Querymap.txt

The resulting file should look e.g. like below:

In [4]:
!head -8 Querymap.txt

WOF01_summer	fastq	../../raw_reads/WOF01_summer-12S_R1.fastq.gz	../../raw_reads/WOF01_summer-12S_R2.fastq.gz
WOF02_summer	fastq	../../raw_reads/WOF02_summer-12S_R1.fastq.gz	../../raw_reads/WOF02_summer-12S_R2.fastq.gz
WOF03_summer	fastq	../../raw_reads/WOF03_summer-12S_R1.fastq.gz	../../raw_reads/WOF03_summer-12S_R2.fastq.gz
WOF04_summer	fastq	../../raw_reads/WOF04_summer-12S_R1.fastq.gz	../../raw_reads/WOF04_summer-12S_R2.fastq.gz
WOF05_summer	fastq	../../raw_reads/WOF05_summer-12S_R1.fastq.gz	../../raw_reads/WOF05_summer-12S_R2.fastq.gz
WOF06_summer	fastq	../../raw_reads/WOF06_summer-12S_R1.fastq.gz	../../raw_reads/WOF06_summer-12S_R2.fastq.gz
WOF07_summer	fastq	../../raw_reads/WOF07_summer-12S_R1.fastq.gz	../../raw_reads/WOF07_summer-12S_R2.fastq.gz
WOF08_summer	fastq	../../raw_reads/WOF08_summer-12S_R1.fastq.gz	../../raw_reads/WOF08_summer-12S_R2.fastq.gz


The 12S amplicon sequenced here is only 106 bp long. Readlength used in the MiSeq run was 2x300bp. Our reads are thus longer than our amplicon and we so expect to find primer/adapter sequences in our reads that need to be removed as part of the raw data processing. 

Specifically, forward reads are expected to contain the reverse complement of the reverse primer plus the reverse Illumina adapter (FA501-FA508; FB501-FB508), and reverse reads will contain reverse complements of the forward primers and adapters (RA701-RA712;RB701-RB712).

The expected sequences have been produced from `12S-adapters.fasta` (see [here](https://github.com/HullUni-bioinformatics/Handley_et_al_2018/blob/master/supplementary_data/12S-adapters.fasta)) into reverse complements `12S-adapters_rc.fasta` and will be used in the trimming algorithm.

In [5]:
!head ../12S-adapters.fasta

In [6]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
for record in SeqIO.parse("../12S-adapters.fasta", "fasta"):
    print record.id + "_rc"
    print record.seq.reverse_complement()

def make_rc_record(record):
    """Returns a new SeqRecord with the reverse complement sequence."""
    return SeqRecord(seq = record.seq.reverse_complement(), \
                 id = record.id + "_rc", \
                 description = "reverse complement")

records = map(make_rc_record, SeqIO.parse("../12S-adapters.fasta", "fasta"))
SeqIO.write(records, "../12S-adapters_rc.fasta", "fasta")

FA501_rc
GGGGTATCTAATCCCAGTCCAATTACCATACGTACGATGTGTAGATCTCGGTGGTCGCCGTATCATT
FA502_rc
GGGGTATCTAATCCCAGTCCAATTACCATACAGATAGTGTGTAGATCTCGGTGGTCGCCGTATCATT
FA503_rc
GGGGTATCTAATCCCAGTCCAATTACCATAACTCGCTAGTGTAGATCTCGGTGGTCGCCGTATCATT
FA504_rc
GGGGTATCTAATCCCAGTCCAATTACCATAACACGCAGGTGTAGATCTCGGTGGTCGCCGTATCATT
FA505_rc
GGGGTATCTAATCCCAGTCCAATTACCATACTCGATGAGTGTAGATCTCGGTGGTCGCCGTATCATT
FA506_rc
GGGGTATCTAATCCCAGTCCAATTACCATACACTCACGGTGTAGATCTCGGTGGTCGCCGTATCATT
FA507_rc
GGGGTATCTAATCCCAGTCCAATTACCATAAGATATCCGTGTAGATCTCGGTGGTCGCCGTATCATT
FA508_rc
GGGGTATCTAATCCCAGTCCAATTACCATAACGGTGTCGTGTAGATCTCGGTGGTCGCCGTATCATT
FB501_rc
GGGGTATCTAATCCCAGTCCAATTACCATATATAGTAGGTGTAGATCTCGGTGGTCGCCGTATCATT
FB502_rc
GGGGTATCTAATCCCAGTCCAATTACCATATAGTAACGGTGTAGATCTCGGTGGTCGCCGTATCATT
FB503_rc
GGGGTATCTAATCCCAGTCCAATTACCATAGTGACTCTGTGTAGATCTCGGTGGTCGCCGTATCATT
FB504_rc
GGGGTATCTAATCCCAGTCCAATTACCATAGTCTCGTAGTGTAGATCTCGGTGGTCGCCGTATCATT
FB505_rc
GGGGTATCTAATCCCAGTCCAATTACCATACGAGACGTGTGTAGATCTCGGTGGTCGCCGTATCATT

40

In [7]:
!head -10 ../12S-adapters_rc.fasta

>FA501_rc reverse complement
GGGGTATCTAATCCCAGTCCAATTACCATACGTACGATGTGTAGATCTCGGTGGTCGCCG
TATCATT
>FA502_rc reverse complement
GGGGTATCTAATCCCAGTCCAATTACCATACAGATAGTGTGTAGATCTCGGTGGTCGCCG
TATCATT
>FA503_rc reverse complement
GGGGTATCTAATCCCAGTCCAATTACCATAACTCGCTAGTGTAGATCTCGGTGGTCGCCG
TATCATT
>FA504_rc reverse complement


__Raw data trimming, removal of adapter sequences and merging of reads__ using the `metaBEAT` pipeline.

In [ ]:
%%bash

metaBEAT_global.py \
-Q Querymap.txt \
--trim_qual 30 \
--trim_adapter ../12S-adapters_rc.fasta \
--trim_minlength 90 \
--merge \
--product_length 110 \
--forward_only \
--read_crop 106 \
-@ haikuilee@gmail.com \
-n 5 -v &> log

In [8]:
cd ../

/home/working/12S


Some stats on the read counts before/after trimming, merging etc. are summarized for you in `metaBEAT_read_stats.csv`.

Next stage of the processing is __chimera detection__ and removal of putative chimeric sequences. We'll do that using `uchime` as implemented in `vsearch`.

In [9]:
!mkdir chimera_detection

In [10]:
cd chimera_detection

/home/working/12S/chimera_detection


Convert reference database from Genbank to fasta format to be used in chimera detection.

Prepare Refmap file, i.e. text file that specifies the location and the format of the reference to be used.

The reference sequences in Genbank format should already be present in the `supplementary_data` directory: `12S_Fish_SATIVA_cleaned_May_2017.gb`.

In [11]:
%%bash

#Write REFmap
for file in $(ls -1 ../../supplementary_data/reference_DBs/* | grep "12S_Fish_SATIVA_cleaned_May_2017.gb$")
do
      echo -e "$file\tgb"
done > REFmap.txt



In [12]:
!cat REFmap.txt

../../supplementary_data/reference_DBs/12S_Fish_SATIVA_cleaned_May_2017.gb	gb


In [ ]:
%%bash

metaBEAT_global.py \
-R REFmap.txt \
-f \
-@ haikuilee@gmail.com

This will produce `refs.fasta`.

Now run chimera detection.

In [ ]:
%%bash


for a in $(cut -f 1 ../trimming/Querymap.txt)
do
    if [ -s ../trimming/$a/$a\_trimmed.fasta ]
    then
        echo -e "\n### Detecting chimeras in $a ###\n"
        mkdir $a
        cd $a
        vsearch --uchime_ref ../../trimming/$a/$a\_trimmed.fasta --db ../refs.fasta \
        --nonchimeras $a-nonchimeras.fasta --chimeras $a-chimeras.fasta &> log
        cd ..

    else
        echo -e "$a is empty"
    fi
done




In [16]:
cd ..

/home/working/12S


Last step is __taxonomic assignment of reads based on a BLAST - LCA approach__ using the metaBEAT pipeline.

In [17]:
!mkdir non-chimeras

In [18]:
cd non-chimeras/

/home/working/12S/non-chimeras


Prepare Querymap and Refmap txt files.

In [22]:
%%bash

#Querymap
for a in $(ls -l ../chimera_detection/ | grep "^d" | perl -ne 'chomp; @a=split(" "); print "$a[-1]\n"')
do
   if [ "$a" != "GLOBAL" ]
   then
      echo -e "$a-nc\tfasta\t../chimera_detection/$a/$a-nonchimeras.fasta"
   fi
done > Querymap.txt

#REFmap
#Write REFmap


#Write REFmap
for file in $(ls -1 ../../supplementary_data/reference_DBs/* | grep "12S_Fish_SATIVA_cleaned_May_2017.gb$")
do
      echo -e "$file\tgb"
done > REFmap.txt

for file in $(ls -1 ../../supplementary_data/reference_DBs/* | grep "RhamphochromisEsox_mt.gb$")
do
      echo -e "$file\tgb"      
      
done >> REFmap.txt

In [ ]:
!head Querymap.txt

In [24]:
!cat REFmap.txt

../../supplementary_data/reference_DBs/12S_Fish_SATIVA_cleaned_May_2017.gb	gb
../../supplementary_data/reference_DBs/RhamphochromisEsox_mt.gb	gb


__Sequence clustering and taxonomic assignment via metaBEAT__

In [ ]:
%%bash

metaBEAT.py \
-Q Querymap.txt \
-R REFmap.txt \
--cluster --clust_match 1 --clust_cov 3 \
--blast --min_ident 1 \
-m 12S -n 5 \
-E -v \
-o 12S-trim_30-merged-nonchimeras-cluster_1c3-blast-min_ident_1.0 &> log

__Results are under the `GLOBAL` folder__